# SIOP 2018 Master Tutorial


The purpose of this Master Tutorial is to teach a bit of data wrangling on a typical social science data set derived from a survey. The purpose is NOT to teach you R or Python. That would be much better handled in a dedicated class on Coursera, Udemy, edX, etc (feel free to email robstilson@gmail.com if you would like recommendations). This tutorial jumps in at around an advanced beginner to intermediate level. Therefore the purpose is more around teaching data wrangling while laying the ground work for a Rosetta Stone type reference between R and Python.

First, we will import the pacakges needed for this script. To do so, we simply wrap the function library() around whatever package we want to load. If the package has not yet been installed in your R envrionment, you will first need to use the function install.packages("Package Name", dependencies = TRUE). The addendum of 'dependencies = TRUE' also installs any packages needed to run the package successfully.



## Load Libraries

In [ ]:
##for importing xlsx files
library(xlsx)

##for creating new variables, recoding variables, filtering/selecting variables
library(dplyr)

##for melting and casting datasets
library(reshape2)

##tidy datasets
library(tidyr)

library(ggplot2)

tryCatch(require(RCurl),finally=utils:::install.packages(pkgs='RCurl',repos='http://cran.r-project.org'));
require(RCurl)

#install.packages('RCurl', dependencies = T)
#library(RCurl)


## Import Data

In [ ]:
##Set working directory

#setwd("C:\\Users\\bufto\\Dropbox (Personal)\\Spring 2018\\SIOP 2018")



As you will see in the Python tutorial, you should NOT use periods in the names you give your datasets. I typically use 'data' and other intuitive versions: data_clean, data_subset, etc.

In [ ]:
##Base r for .csv file import

#data <- read.csv("SIOP Data Wrangling Master Tutorial Data Set.csv")

library(RCurl)
x <- getURL("https://raw.githubusercontent.com/RobStilson/SIOP_2018_Master_Tutorial/master/SIOP%20Data%20Wrangling%20Master%20Tutorial%20Data%20Set%20REGEX.csv")
data <- read.csv(text = x)
head(data)



In [ ]:
##Load libraries for importing different file types -- .xlsx

library(xlsx)
data2 <- read.xlsx("SIOP Data Wrangling Master Tutorial Data Set.xlsx", sheetName = "SIOP Data Wrangling Master Tuto")
data2

In [ ]:
##Load libraries for importing different file types -- .sav

library(foreign)
data3 <- read.spss("SIOP Data Wrangling Master Tutorial Data Set.sav", to.data.frame=TRUE)
data3

In [ ]:
##Load libraries for importing different file types -- .dat
library(foreign)
data4 <- read.table("SIOP Data Wrangling Master Tutorial Data Set.dat",sep= "\t", header = TRUE)
data4

## Explore the data



Now we are going to explore the data in a number of different ways, similar to ways that you might be interested in examining data when you're processing your own organizational data. First, we will use head() to look at the first 5 rows of our dataset. Note that you can tailor the number in the function to look at a different numbers of rows.

In [ ]:
head(data,5)

Next, we will look at the structure of the data, which outlines each of the variables in your dataset, the types of variables, and some examples of the observations within each variable

In [ ]:
str(data)



I also use name() quite a bit, which provides you with a quick reminder of your variable names. This funciton is useful for renaming variables, using them in analyses, and subsetting data.

In [ ]:
names(data)

## Cleaning the string data

There are a host of issues that can plague string data that make data cleaning and analysis a nightmare. The Python script goes into detail about Regular Expressions (regex), which are conventions that can be utilized to identify and manipulate patterns of text within variable names and observations. I will show some examples of regex applications and some brute force code using the gsub() function. As you will learn as you become a more experienced R user, there are often many different ways to accomplish the same task in R. If you have the time (operative word being 'if'), try to optimize the most efficient script possible, otherwise, just do whatever works.

### Regular Expressions

In [ ]:
#Replacing spaces with underscores
names(data) <- gsub(x = names(data),
                    pattern = " ",
                    replacement = "_")

In [ ]:
#Replace ":" with "_" in variable names
names(data) <- gsub(x = names(data),
                    pattern = "\\:",
                    replacement = "_")


In [ ]:
#Replacing em-dash
names(data) <- gsub(x = names(data),
                    pattern = '.[\u2013].', #"-" aka "em-dash" From: https://stat.ethz.ch/pipermail/r-help/2017-April/438139.html
                    replacement = ".")

In [ ]:
#removing double quotes and curly double quotes
names(data) <- gsub(x = names(data),
                    pattern = "[\u201C\u201D\u201E\u201F\u2033\u2036]",
                    replacement = "")

In [ ]:
#Check columns to make sure code worked
colnames(data)

In [ ]:
##Remove text before last period for MC items

names(data)[9:59] <- gsub(x = names(data)[9:59], 
                    pattern = ".*\\.", 
                    replacement = "")

In [ ]:
colnames(data)


### Brute force variable name changes

In [ ]:
##brute force variable name change, Method #1

names(data) <- gsub(x = names(data),
                    pattern = "On.a.scale.of.zero.to.ten..how.likely.is.it.that.you.would.recommend.Company.to.friends.as.a.great.place.to.work.",
                   replacement = "NPS_Score")



In [ ]:
##brute force variable name change, Method #2

data <- rename(data, 
                    OEQ_1 = 
                       `What.2.3.things.do.you.value.most.about.working.at.Company.`,
                     OEQ_2 = 
                       `What.2.3.things.should.Company.begin.to.do.`,
                     OEQ_3 = 
                       `What.2.3.things.should.Company.stop.doing.`,
                     OEQ_4 = 
                       `Please.provide.suggestions.for.ongoing.improvement.to.the.performance.feedback.process.`)



In [ ]:
##Check to see if it worked

names(data)

### Removing columns

Here we remove the Respondent_IP column because we don't need it. To remove a column in R, I recommend using the select() function within the dplyr family. It allows you to select (and remove) columns based on the column name, without parentheses, which I recommend doing rather than specifying the column number. That way, if your change the order of columns in your original data file, you won't lose a column of interest. 

Note the '-' indicates removing a column, you can also specify columns to keep in this function by not including the minus sign.

In [ ]:
data <- data %>% 
    select(-Respondent.IP, -Spaces, -Colons, -EmDash,-Quotes)
names(data)


## Tidy Data (e.g. Wide vs. Tall, Melted vs. Casted, Gathered, vs. Spread)

[Tidy Data](https://vita.had.co.nz/papers/tidy-data.pdf) by Hadley Wickham is an excellent read on how your data frames should be set up. The way I try to explain it is that in I/O Psychology we typically work with 1 row per person. This leads to very wide data sets as more variables are added. Especially with multiple metrics over multiple years (12 metrics by 3 years of montly data becomes 432 columns of data). A tall data set on the other hand is multiple rows per person with fewer columns. This also allows me to write one line of code to get the mean, SD, etc. for all of those variables instead of individually coding each of them. It is probably better to explain this visually, so I will send you to the excellent tutorial by Sean C. Anderson [here](http://seananderson.ca/2013/10/19/reshape/).

### Melting data in R

To melt a dataset in R, I recommend using the reshape2 package with the melt() function. 

I also recommend using new naming conventions for your dataset when you start manipulating the structure of your dataset to keep track of the changes. This habit also allows you to correct mistakes in your code more efficiently. 

In [ ]:
names(data)

In [ ]:
data_melt <- melt(data, id.vars = c("Number", 
                 "First.Name",
                 "Last.Name",
                 "Email.Address",
                 "Department",
                 "Division",
                 "Completed.On"))

Check the head of the newly melted data to make sure everything looks correct. Notice that the variables that were melted will be stored in two new columns: variable and value. The variable column holds the variable names and the value column holds the values of those variables. 

You may see a Warning Message after melting your data, as we did here, noting that the attributes of the dataset were not identical and were therefore dropped. This message will occur when you melt different kinds of variables (e.g., factors, characters, numerical values) together. This should not affect the actual values stored in the value column. 

In [ ]:
head(data_melt, 10)

### Splitting Columns in R

Now with our data in tall form, we are going to split the variable column into Area (for broad subscale/topic area) and Item (for the specific item in each subscale). There are several ways to split columns in R, including the base R split() function and the str_split function from the stringr library. However, in keeping with the Tidyverse theme of the presentation, I will show a method using the separate() function within the dplyr library.

In [ ]:
##separating the variable column on the '_' into Area and Item

data_melt <- separate(data_melt, variable, into = c("Area", "Item"), sep ="_", remove = FALSE)

In [ ]:
##Check to make sure you made your new columns

names(data_melt)

In [ ]:
##Also check the head of the data to make sure the observations look okay

head(data_melt, 10)

### Creating Aliases for data groups

Creating 'Aliases' for data groups makes it easier to roll data up and serves as a proxy for those (including myself) who came from SPSS and got used to labels for long item names. Since neither Python or R do that, here is my work around.

#### Finding unique data in tall format

We are going to find the unique (i.e., distinct) variable names in our melted data frame using unique(). One way I might use this is to then create a "Helper" file in Excel where I paste the results of the unique Items within the variable column and then use concatenate with the necessary programming language. I will often do this if I need to create 20 or more similar lines of code for renaming, ifelse, etc.

In [ ]:
unique(data_melt$variable)

Now we will actually create the aliases in the data frame. There may be a better way to do this, but I will use an nested structure of ifelse statements. First, though, we create an empty column of data called "Alias" that we will populate through the ifelse statements. 

In [ ]:
## create empty Alias column

data_melt$Alias <- NA
names(data_melt)

Below is the nested ifelse function. Note that there is a limit of 50 nests within this function, so I separate them out. 

In [ ]:
data_melt <- data_melt %>%
  mutate(Alias =
ifelse(variable == "ECS_Q1", "I am proud to work at Company." , 
ifelse(variable == "ECS_Q2", "I feel appreciated for the work I do." , 
ifelse(variable == "ECS_Q3", "I have what I need to do my job" , 
ifelse(variable == "ECS_Q4", "My job is good." ,       
ifelse(variable == "ECS_Q5", "I like my work." , 
ifelse(variable == "ECS_Q6", "Works is fun.",        
ifelse(variable == "WEC_Q1", "I feel free to say what I want." , 
ifelse(variable == "WEC_Q2", "Ideas are cool." , 
ifelse(variable == "WEC_Q3", "We go to happy hours." , 
ifelse(variable == "WEC_Q4","I know why we do things." ,
ifelse(variable == "WEC_Q5", "Things are fair." , 
ifelse(variable == "CB_Q1","I know how much I make." , 
ifelse(variable == "CB_Q2", "I get paid enough." , 
ifelse(variable == "CB_Q3", "I am happy with my pay" , 
ifelse(variable == "CB_Q4", "I get my more money if I work hard" , 
ifelse(variable == "PM_Q1", "My boss knows stuff." , 
ifelse(variable == "PM_Q2", "My boss helps me." , 
ifelse(variable == "PM_Q3", "My boss is helpful." , 
ifelse(variable == "PM_Q4", "My boss does his/her job." , 
ifelse(variable == "PM_Q5", "My other boss is helpful." , 
ifelse(variable == "PM_Q6", "My other buss helps me." ,     
ifelse(variable == "PM_Q7", "My other boss does his/her job." , 
ifelse(variable == "CDT_Q1", "Company taught me how to do my job well." , 
ifelse(variable == "CDT_Q2", "Company gives me opportunities." , 
ifelse(variable == "CDT_Q3", "I can grow at Company." , 
ifelse(variable == "CDT_Q4", "I want to grow at Company." , 
ifelse(variable == "CDT_Q5","Everyone has a fair shot at growth at Company." , 
ifelse(variable == "CDT_Q6", "Company has clear promotion standards." ,   
ifelse(variable == "COMM_Q1", "I know when Company changes stuff." , 
ifelse(variable == "COMM_Q2", "I get info about how to do my job well." , 
ifelse(variable == "COMM_Q3", "Corp and the people have transparency." , 
ifelse(variable == "COMM_Q4", "I know why Company changes stuff." ,        
ifelse(variable == "TE_Q1", "At Company we work effectively as a team." , 
ifelse(variable == "TE_Q2", "At Company, I can use my strengths to succeed" , 
ifelse(variable == "TE_Q3", "We have good teamwork." , 
ifelse(variable == "TE_Q4", "I have shared goals with my co-workers." , 
ifelse(variable == "TE_Q5", "My team lets me know when I mess up." , 
ifelse(variable == "JS_Q1", "I don't have too much work." , 
ifelse(variable == "JS_Q2", "I have a good work-life balance." , 
ifelse(variable == "JS_Q3", "My work environment is fun." , 
NA)))))))))))))))))))))))))))))))))))))))))

###split because ifelse nesting can only handle 50 statements at a time    
data_melt <- data_melt %>%
  mutate(Alias =      
ifelse(variable == "CI_Q1", "Our customers rock" , 
ifelse(variable == "CI_Q2", "I like working hard for our customers." , 
ifelse(variable == "CI_Q3", "Customer service provides meaning to my work." , 
ifelse(variable == "CI_Q4", "Company cares about our customers." , 
ifelse(variable == "STRAT_Q1", "I am confidence the company is going in a good direction." , 
ifelse(variable == "STRAT_Q2", "Company has a good plan for the future." , 
ifelse(variable == "STRAT_Q3", "I know how I contribute to Company's goals" , 
ifelse(variable == "NPS_Score", "On a scale of zero to ten how likely is it that you would recommend Company to friends as a great place to work" , 
ifelse(variable == "OEQ_1", "What do you like about working at Company?", 
ifelse(variable == "OEQ_2", "What do you not like about working at Company", 
ifelse(variable == "OEQ_3", "What could Company improve?", 
ifelse(variable == "OEQ_4", "Any other info for Company?", 
       Alias)))))))))))))

In [ ]:
head(data_melt, 10)

In [ ]:
##See full data
unique(data_melt$Alias)

data_melt

### Creating Alias_Area

Here we are going to create an Alias Area to further roll up the data. This allows us to simiply grab all of the ECS items by choosing the ECS Alias Area rather than ECS1, ECS2, etc.

We will then write to a .csv to have a look at the data because sometimes that is just easier with a spread sheet.

In [ ]:
##Creating Alias_Area

data_melt$Alias_Area <- NA
unique(data_melt$Area)

In [ ]:
##Creating Alias_Area

data_melt <- data_melt %>%
  mutate(Alias_Area = 
ifelse(Area == "ECS","Commitment and Satisfaction" , 
ifelse(Area == "WEC", "Workplace Environment and Culture" , 
ifelse(Area == "CB", "Compensation and Benefits" , 
ifelse(Area == "PM", "Performance Management" , 
ifelse(Area == "CDT", "Career Development and Training" , 
ifelse(Area == "COMM", "Communications" , 
ifelse(Area == "TE", "Team Effectiveness" , 
ifelse(Area == "JS" , "Job Stress",
ifelse(Area == "CI", "Company Image" , 
ifelse(Area == "STRAT", "Strategy" ,     
ifelse(Area == "NPS" , "NPS",
ifelse(Area == "OEQ" , "OEQ" ,
        NA)))))))))))))

In [ ]:
##Check out the new observations within Alias_Area

unique(data_melt$Alias_Area)

## Recoding values in R

Below we will recode the values of Strongly Disagree - Strongly Agree into numeric values so we can do mean, SD, etc.

I like to use the combination of the mutate function in dplyr to create new variables or change the values of existing variables and nested ifelse statements.

In [ ]:
data_melt <- data_melt %>% 
  mutate(response = ifelse(value == "Strongly Disagree", 1,
                 ifelse(value == "Disagree", 2,
                 ifelse(value == "Somewhat Disagree", 3,
                 ifelse(value == "Somewhat Agree", 4,
                 ifelse(value == "Agree", 5,
                 ifelse(value == "Strongly Agree", 6,
                    value)))))))
unique(data_melt$response)    

## Separate out OEQ (Text) data

We don't have any actual Text data under the Open Ended Question (OEQ) variables, but I wanted to show you how to do that because it is very common to have comments in survey data.

One thing to keep in mind with tall data is that when dealing with mixed data types (numeric and character) is that once they are in the same column (value), then the entire column will be switched over to character (text) values. This remains even after you seperate out the string values. We can check the data types with the class() or str() functions.

In [ ]:
str(data_melt)

In this case we won't have to convert it, but we will go through the steps anyway since that is more realistic for data you will typically encounter. 

In [ ]:
data_melt_num <- data_melt %>%
    filter(Area != "OEQ" &
          Area != "NPS")

In [ ]:
unique(data_melt_num$Area)

In [ ]:
##convert 'response' to numeric variable type in data_melt_num

data_melt_num$response <- as.numeric(data_melt_num$response)
class(data_melt_num$response)
head(data_melt,5)

## Summarizing (group_by) data in R

Since we created our Area earlier, we can now group by that variable to take the mean. We will then sort it within the same line. We also show you how to round to 2 decimals to keep the data neat and readable.

In [ ]:
##Grouping into 'Area' and taking the mean

Topic_Area_Means <- data_melt_num %>%
group_by(Area) %>%
summarize(Area_Mean = round(mean(response,na.rm = TRUE),2))
Topic_Area_Means

## Visualizing the data (Bar Chart) in R

Now we will create a bar chart using another package called ggplot2. ggplot2 is great because there are innumerable ways that you can tailor and fine-tune your plots, although that can also be its downside. The ggplot2 cheat sheet, found [here](https://www.rstudio.com/wp-content/uploads/2015/03/ggplot2-cheatsheet.pdf) and the ggplot2 cookbook ([preview here](http://www.cookbook-r.com/Graphs/)) are great resources to get started in ggplot2.

I recommend saving any ggplot2 code that you create in a single script so that you can reference and tweak it every time you're making a new regression plot or bar chart, rather than reinventing the wheel every time.

In [ ]:
##Create a pretty theme

windowsFonts(Calibri=windowsFont("Calibri"))

Hor_Bar_LightBlue_Theme <-theme(
text = element_text(family = "Calibri"),
axis.title.x = element_blank(),
axis.title.y = element_blank(),
axis.line = element_blank(),
#panel.border = element_blank(),
#panel.grid=element_blank(),
axis.ticks = element_blank(),
plot.title=element_text(size=20, face="bold",hjust=0.6),
plot.subtitle=element_text(size=15,face=c("bold","italic"),hjust=0.6),
axis.text.x=element_text(size=15),#element_blank(),
axis.text.y=element_text(size=20),
legend.position = "none",
panel.spacing=unit(2,"cm"),
    panel.background = element_rect(fill = "white",colour = NA), # or element_blank()
    panel.grid.minor = element_line(color = "gray90", size = 0.20),
    panel.grid.minor.y = element_blank(),
    panel.grid.major = element_line(color = "gray90", size = 0.20),
    panel.grid.major.y = element_blank()#removes horizontal lines
    # plot.background = element_rect(fill = "transparent",colour = NA)
)

In [ ]:
##Horizontal Bar Chart

hbar_Overall_Area <- ggplot(Topic_Area_Means, aes(Area, y=Area_Mean, fill=Area)) +
  geom_bar(stat='identity') + #Light Blue
  geom_text(aes(x=Area, y=Area_Mean, label=sprintf("%0.2f", round(Area_Mean, digits = 2))),hjust = -0.1, color="#4D4D4D", size = 6, fontface = "bold") + 
  Hor_Bar_LightBlue_Theme + 
  coord_flip(ylim = c(1,6)) + scale_y_continuous(breaks=seq(1,6,1), position = "right") #Add this to make 1,2,3,4,5,6 appear on axis 
#print it!
print(hbar_Overall_Area)

